In [ ]:


import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import torch
import torch.nn as nn
from torch.nn import functional as F
from matplotlib import pyplot as plt
from PIL import Image
from torchvision import transforms as tsf
import csv
%pylab inline

Populating the interactive namespace from numpy and matplotlib


BAM

In [ ]:
import math

class Flatten(nn.Module):
    def forward(self, x):
        return x.view(x.size(0), -1)
class ChannelGate(nn.Module):
    def __init__(self, gate_channel, reduction_ratio=16, num_layers=1):
        super(ChannelGate, self).__init__()
        #self.gate_activation = gate_activation
        self.gate_c = nn.Sequential()
        self.gate_c.add_module( 'flatten', Flatten() )
        gate_channels = [gate_channel]
        gate_channels += [gate_channel // reduction_ratio] * num_layers
        gate_channels += [gate_channel]
        for i in range( len(gate_channels) - 2 ):
            self.gate_c.add_module( 'gate_c_fc_%d'%i, nn.Linear(gate_channels[i], gate_channels[i+1]) )
            self.gate_c.add_module( 'gate_c_bn_%d'%(i+1), nn.BatchNorm1d(gate_channels[i+1]) )
            self.gate_c.add_module( 'gate_c_relu_%d'%(i+1), nn.ReLU() )
        self.gate_c.add_module( 'gate_c_fc_final', nn.Linear(gate_channels[-2], gate_channels[-1]) )
    def forward(self, in_tensor):
        avg_pool = F.avg_pool2d( in_tensor, in_tensor.size(2), stride=in_tensor.size(2) )
        return self.gate_c( avg_pool ).unsqueeze(2).unsqueeze(3).expand_as(in_tensor)

class SpatialGate(nn.Module):
    def __init__(self, gate_channel, reduction_ratio=16, dilation_conv_num=2, dilation_val=4):
        super(SpatialGate, self).__init__()
        self.gate_s = nn.Sequential()
        self.gate_s.add_module( 'gate_s_conv_reduce0', nn.Conv2d(gate_channel, gate_channel//reduction_ratio, kernel_size=1))
        self.gate_s.add_module( 'gate_s_bn_reduce0',	nn.BatchNorm2d(gate_channel//reduction_ratio) )
        self.gate_s.add_module( 'gate_s_relu_reduce0',nn.ReLU() )
        for i in range( dilation_conv_num ):
            self.gate_s.add_module( 'gate_s_conv_di_%d'%i, nn.Conv2d(gate_channel//reduction_ratio, gate_channel//reduction_ratio, kernel_size=3, \
						padding=dilation_val, dilation=dilation_val) )
            self.gate_s.add_module( 'gate_s_bn_di_%d'%i, nn.BatchNorm2d(gate_channel//reduction_ratio) )
            self.gate_s.add_module( 'gate_s_relu_di_%d'%i, nn.ReLU() )
        self.gate_s.add_module( 'gate_s_conv_final', nn.Conv2d(gate_channel//reduction_ratio, 1, kernel_size=1) )
    def forward(self, in_tensor):
        return self.gate_s( in_tensor ).expand_as(in_tensor)
class BAM(nn.Module):
    def __init__(self, gate_channel):
        super(BAM, self).__init__()
        self.channel_att = ChannelGate(gate_channel)
        self.spatial_att = SpatialGate(gate_channel)
    def forward(self,in_tensor):
        att = 1 + F.sigmoid( self.channel_att(in_tensor) * self.spatial_att(in_tensor) )
        return att * in_tensor


CBAM

In [ ]:
class BasicConv(nn.Module):
    def __init__(self, in_planes, out_planes, kernel_size, stride=1, padding=0, dilation=1, groups=1, relu=True, bn=True, bias=False):
        super(BasicConv, self).__init__()
        self.out_channels = out_planes
        self.conv = nn.Conv2d(in_planes, out_planes, kernel_size=kernel_size, stride=stride, padding=padding, dilation=dilation, groups=groups, bias=bias)
        self.bn = nn.BatchNorm2d(out_planes,eps=1e-5, momentum=0.01, affine=True) if bn else None
        self.relu = nn.ReLU() if relu else None

    def forward(self, x):
        x = self.conv(x)
        if self.bn is not None:
            x = self.bn(x)
        if self.relu is not None:
            x = self.relu(x)
        return x

class CFlatten(nn.Module):
    def forward(self, x):
        return x.view(x.size(0), -1)

class CChannelGate(nn.Module):
    def __init__(self, gate_channels, reduction_ratio=16, pool_types=['avg', 'max']):
        super(CChannelGate, self).__init__()
        self.gate_channels = gate_channels
        self.mlp = nn.Sequential(
            Flatten(),
            nn.Linear(gate_channels, gate_channels // reduction_ratio),
            nn.ReLU(),
            nn.Linear(gate_channels // reduction_ratio, gate_channels)
            )
        self.pool_types = pool_types
    def forward(self, x):
        channel_att_sum = None
        for pool_type in self.pool_types:
            if pool_type=='avg':
                avg_pool = F.avg_pool2d( x, (x.size(2), x.size(3)), stride=(x.size(2), x.size(3)))
                channel_att_raw = self.mlp( avg_pool )
            elif pool_type=='max':
                max_pool = F.max_pool2d( x, (x.size(2), x.size(3)), stride=(x.size(2), x.size(3)))
                channel_att_raw = self.mlp( max_pool )
            elif pool_type=='lp':
                lp_pool = F.lp_pool2d( x, 2, (x.size(2), x.size(3)), stride=(x.size(2), x.size(3)))
                channel_att_raw = self.mlp( lp_pool )
            elif pool_type=='lse':
                # LSE pool only
                lse_pool = logsumexp_2d(x)
                channel_att_raw = self.mlp( lse_pool )

            if channel_att_sum is None:
                channel_att_sum = channel_att_raw
            else:
                channel_att_sum = channel_att_sum + channel_att_raw

        scale = F.sigmoid( channel_att_sum ).unsqueeze(2).unsqueeze(3).expand_as(x)
        return x * scale

def logsumexp_2d(tensor):
    tensor_flatten = tensor.view(tensor.size(0), tensor.size(1), -1)
    s, _ = torch.max(tensor_flatten, dim=2, keepdim=True)
    outputs = s + (tensor_flatten - s).exp().sum(dim=2, keepdim=True).log()
    return outputs

class ChannelPool(nn.Module):
    def forward(self, x):
        return torch.cat( (torch.max(x,1)[0].unsqueeze(1), torch.mean(x,1).unsqueeze(1)), dim=1 )

class CSpatialGate(nn.Module):
    def __init__(self):
        super(CSpatialGate, self).__init__()
        kernel_size = 7
        self.compress = ChannelPool()
        self.spatial = BasicConv(2, 1, kernel_size, stride=1, padding=(kernel_size-1) // 2, relu=False)
    def forward(self, x):
        x_compress = self.compress(x)
        x_out = self.spatial(x_compress)
        scale = F.sigmoid(x_out) # broadcasting
        return x * scale

class CBAM(nn.Module):
    def __init__(self, gate_channels, reduction_ratio=16, pool_types=['avg', 'max'], no_spatial=False):
        super(CBAM, self).__init__()
        self.CChannelGate = CChannelGate(gate_channels, reduction_ratio, pool_types)
        self.no_spatial=no_spatial
        if not no_spatial:
            self.CSpatialGate = CSpatialGate()
    def forward(self, x):
        x_out = self.CChannelGate(x)
        if not self.no_spatial:
            x_out = self.CSpatialGate(x_out)
        return x_out

In [ ]:
import glob
import os
import numpy as np

def csv_reader(path):
    with open(path, "r") as f:
        breast = list(csv.reader(f))
    return breast
def get_datas(image_dir,suffix=".png"):
   
    image_paths = []
    labels = []
    train=csv_reader("aptos2019-blindness-detection/train.csv")
    for i in train[1:]:
        #y_this=[0.0,0.0,0.0,0.0,0.0]
        #y_this[int(i[1])]=1.0
        image_paths.append(image_dir+'/'+i[0]+'.png')
        labels.append(int(i[1]))
        
    return image_paths,labels

def show_batch(img_paths):
    '''
   
    '''
    randomed = []

    if len(img_paths) <= 25:
        randomed = list(range(0, len(img_paths)))
    else:
        for i in range(0, len(img_paths)):
            random = np.random.randint(0, len(img_paths))
            if random not in randomed:
                randomed.append(random)
            if len(randomed) == 25:
                break
    
    plt.figure(dpi=224)
    for i in range(len(randomed)):
        img = Image.open(img_paths[randomed[i]])
        plt.subplot(5, 5, i+1) 
        plt.imshow(img)
        plt.xticks([])
        plt.yticks([])
    plt.show()

In [ ]:
cd drive

/content/drive


In [ ]:
cd MyDrive

/content/drive/MyDrive


In [ ]:
img_dir = os.path.abspath('aptos2019-blindness-detection/train_images')
shuffix = ".png"
img_paths, labels = get_datas(img_dir, suffix=shuffix)


In [ ]:
import torch.nn as nn
import math
import torch.utils.model_zoo as model_zoo


__all__ = ['ResNet', 'resnet18', 'resnet34', 'resnet50', 'resnet101',
           'resnet152']


model_urls = {
    'resnet18': 'https://download.pytorch.org/models/resnet18-5c106cde.pth',
    'resnet34': 'https://download.pytorch.org/models/resnet34-333f7ec4.pth',
    'resnet50': 'https://download.pytorch.org/models/resnet50-19c8e357.pth',
    'resnet101': 'https://download.pytorch.org/models/resnet101-5d3b4d8f.pth',
    'resnet152': 'https://download.pytorch.org/models/resnet152-b121ed2d.pth',
}


def conv3x3(in_planes, out_planes, stride=1):
    "3x3 convolution with padding"
    return nn.Conv2d(in_planes, out_planes, kernel_size=3, stride=stride,
                     padding=1, bias=False)


class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, inplanes, planes, stride=1, downsample=None, use_cbam=False):
        super(BasicBlock, self).__init__()
        self.conv1 = conv3x3(inplanes, planes, stride)
        self.bn1 = nn.BatchNorm2d(planes)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = conv3x3(planes, planes)
        self.bn2 = nn.BatchNorm2d(planes)
        self.downsample = downsample
        self.stride = stride
        if use_cbam:
            self.cbam = CBAM( planes, 16 )
        else:
            self.cbam = None
    def forward(self, x):
        residual = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)

        if self.downsample is not None:
            residual = self.downsample(x)
        if not self.cbam is None:
            out = self.cbam(out)
        out += residual
        out = self.relu(out)

        return out


class Bottleneck(nn.Module):
    expansion = 4

    def __init__(self, inplanes, planes, stride=1, downsample=None, use_cbam=False):
        super(Bottleneck, self).__init__()
        self.conv1 = nn.Conv2d(inplanes, planes, kernel_size=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=stride,
                               padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)
        self.conv3 = nn.Conv2d(planes, planes * 4, kernel_size=1, bias=False)
        self.bn3 = nn.BatchNorm2d(planes * 4)
        self.relu = nn.ReLU(inplace=True)
        self.downsample = downsample
        self.stride = stride
        if use_cbam:
            self.cbam = CBAM( planes * 4, 16 )
        else:
            self.cbam = None
        
    def forward(self, x):
        residual = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)
        out = self.relu(out)

        out = self.conv3(out)
        out = self.bn3(out)

        if self.downsample is not None:
            residual = self.downsample(x)
        if not self.cbam is None:
            out = self.cbam(out)
        out += residual
        out = self.relu(out)

        return out


class ResNet(nn.Module):

    def __init__(self, block, layers, num_classes=1000, att_type=None):
        self.inplanes = 64
        super(ResNet, self).__init__()
        self.conv1 = nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=3,
                               bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        if att_type=='BAM':
            self.bam1 = BAM(64*block.expansion)
            self.bam2 = BAM(128*block.expansion)
            self.bam3 = BAM(256*block.expansion)
        else:
            self.bam1, self.bam2, self.bam3 = None, None, None
        self.layer1 = self._make_layer(block, 64, layers[0], att_type=att_type)
        self.layer2 = self._make_layer(block, 128, layers[1], stride=2, att_type=att_type)
        self.layer3 = self._make_layer(block, 256, layers[2], stride=2, att_type=att_type)
        self.layer4 = self._make_layer(block, 512, layers[3], stride=2, att_type=att_type)
        self.avgpool = nn.AdaptiveAvgPool2d(1)
        self.fc = nn.Linear(512 * block.expansion, 1000)
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                n = m.kernel_size[0] * m.kernel_size[1] * m.out_channels
                m.weight.data.normal_(0, math.sqrt(2. / n))
            elif isinstance(m, nn.BatchNorm2d):
                m.weight.data.fill_(1)
                m.bias.data.zero_()

    def _make_layer(self, block, planes, blocks, stride=1,att_type=None):
        downsample = None
        if stride != 1 or self.inplanes != planes * block.expansion:
            downsample = nn.Sequential(
                nn.Conv2d(self.inplanes, planes * block.expansion,
                          kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(planes * block.expansion),
            )

        layers = []
        layers.append(block(self.inplanes, planes, stride, downsample, use_cbam=att_type=='CBAM'))
        self.inplanes = planes * block.expansion
        for i in range(1, blocks):
            layers.append(block(self.inplanes, planes, use_cbam=att_type=='CBAM'))

        return nn.Sequential(*layers)

    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)

        x = self.layer1(x)
        if not self.bam1 is None:
            x = self.bam1(x)
        x = self.layer2(x)
        if not self.bam2 is None:
            x = self.bam2(x)
        x = self.layer3(x)
        if not self.bam3 is None:
            x = self.bam3(x)
        feat = self.layer4(x)

        x = self.avgpool(feat)
        x = x.view(x.size(0), -1)
        x = self.fc(x)

        return feat, x


def resnet18(pretrained=False, att_type=None):
    """Constructs a ResNet-18 model.
    Args:
        pretrained (bool): If True, returns a model pre-trained on ImageNet
    """
    model = ResNet(BasicBlock, [2, 2, 2, 2], att_type=att_type)
    if pretrained:
        save_model = model_zoo.load_url(model_urls['resnet18'])
        model_dict =  model.state_dict()
        state_dict = {k:v for k,v in save_model.items() if k in model_dict.keys()}
        model_dict.update(state_dict)
        model.load_state_dict(model_dict)
    return model


def resnet34(pretrained=False, att_type=None):
    """Constructs a ResNet-34 model.
    Args:
        pretrained (bool): If True, returns a model pre-trained on ImageNet
    """
    model = ResNet(BasicBlock, [3, 4, 6, 3], att_type=att_type)
    if pretrained:
        model.load_state_dict(model_zoo.load_url(model_urls['resnet34']))
    return model


def resnet50(pretrained=False, **kwargs):
    """Constructs a ResNet-50 model.
    Args:
        pretrained (bool): If True, returns a model pre-trained on ImageNet
    """
    model = ResNet(Bottleneck, [3, 4, 6, 3], **kwargs)
    if pretrained:
        model.load_state_dict(model_zoo.load_url(model_urls['resnet50']))
    return model


def resnet101(pretrained=False, att_type=None):
    """Constructs a ResNet-101 model.
    Args:
        pretrained (bool): If True, returns a model pre-trained on ImageNet
    """
    model = ResNet(Bottleneck, [3, 4, 23, 3], att_type=att_type)
    if pretrained:
        save_model = model_zoo.load_url(model_urls['resnet101'])
        model_dict =  model.state_dict()
        state_dict = {k:v for k,v in save_model.items() if k in model_dict.keys()}
        model_dict.update(state_dict)
        model.load_state_dict(model_dict)
    return model


def resnet152(pretrained=False, att_type=None):
    """Constructs a ResNet-152 model.
    Args:
        pretrained (bool): If True, returns a model pre-trained on ImageNet
    """
    model = ResNet(Bottleneck, [3, 8, 36, 3], att_type=att_type)
    if pretrained:
        save_model = model_zoo.load_url(model_urls['resnet152'])
        model_dict =  model.state_dict()
        state_dict = {k:v for k,v in save_model.items() if k in model_dict.keys()}
        model_dict.update(state_dict)
        model.load_state_dict(model_dict)
    return model

In [ ]:
from torch.utils.data import Dataset

class EyeDataset(Dataset):
    def __init__(self, img_paths, labels, gray=False, transform=None):
        super(EyeDataset, self).__init__()
        self.img_paths = img_paths
        self.labels = labels
        self.gray = gray
        self.transform = transform
        self.length = len(img_paths)
    
    def __len__(self):
        return self.length
    
    def __getitem__(self, index):
        img_path = self.img_paths[index]
        label = self.labels[index]
        img = Image.open(img_path)
        if self.gray:
            img.convert("L")
        else:
            img.convert("RGB")
        
        if self.transform is not None:
            img = self.transform(img)
        
        return img, label

In [ ]:

from torch.utils.data.sampler import WeightedRandomSampler
from sklearn.model_selection import train_test_split
train_paths, val_paths, train_labels, val_labels = train_test_split(img_paths, labels, test_size=0.3, random_state=0, stratify=labels)

img_size = 224
train_transform = tsf.Compose([
    tsf.Resize((img_size, img_size)),
    tsf.RandomHorizontalFlip(),
    tsf.RandomVerticalFlip(),
    tsf.ToTensor(),
    tsf.Normalize(mean=[0.5, 0.5, 0.5], std=[0.25, 0.25, 0.25])
])
val_transform = tsf.Compose([
    tsf.Resize((img_size, img_size)),
    tsf.ToTensor(),
    tsf.Normalize(mean=[0.5, 0.5, 0.5], std=[0.25, 0.25, 0.25])
])
train_dataset =EyeDataset(train_paths, torch.tensor(train_labels), transform=train_transform)
val_dataset = EyeDataset(val_paths, torch.tensor(val_labels), transform=val_transform)

In [ ]:

from torch.utils.data import DataLoader
wights=[]
train_time=[1,5,2,9,6]
for i in train_labels:
    wights.append(train_time[int(i)])
sampler = WeightedRandomSampler(wights, len(wights),replacement=True)
train_loader = DataLoader(train_dataset, shuffle=False, batch_size=8, num_workers=4, sampler=sampler) #备注：在windows系统中多线程可能存在问题，所以设置num_workers为0
val_loader = DataLoader(val_dataset, shuffle=False, batch_size=1, num_workers=4)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [ ]:
def train_for(x,label,opt,model,losses,total,number,count):
    for _ in range(number):
        x = x.to(device, dtype=torch.float32)
        label = label.to(device, dtype=torch.long)
        batch = x.size(0)
        total += batch
        opt.zero_grad()
        _,pred = model(x)
        loss = criterion(pred, label)
        loss.backward()
        pred = torch.max(pred, dim=1)[1]
        for i in range(len(pred)):
            if int(pred[i])==int(label[i]):
                count+=1
        if total%128==0:
            print(loss)
            print(total)
            print("acc: "+str(count/128))
            count=0
        opt.step() 
        losses += loss.item() * batch
    return opt,model,losses,total,count

In [ ]:
import random
model = resnet101(pretrained=True,att_type='BAM')
#model=DenseNet121()
model.fc= nn.Linear(2048, 5)
opt = torch.optim.SGD(model.parameters(),lr=0.004, momentum=0.9, nesterov=True)
scheduler = torch.optim.lr_scheduler.StepLR(opt, step_size = 6, gamma = 0.1, last_epoch=-1)
criterion = nn.CrossEntropyLoss()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
for __ in range(1):
    for epoch in range(1):
        losses = 0.0
        total = 0
        count=0
        corrects = 0
        model.train()
        for x, label in train_loader:
            opt,model,losses,total,count=train_for(x,label,opt,model,losses,total,1,count)
        print("avg loss: "+str(losses))
        scheduler.step()
torch.save(model,"./model.pkl")
torch.save(model.state_dict(),"model.pth")

Downloading: "https://download.pytorch.org/models/resnet101-5d3b4d8f.pth" to /root/.cache/torch/hub/checkpoints/resnet101-5d3b4d8f.pth


  0%|          | 0.00/170M [00:00<?, ?B/s]

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:1806: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


tensor(1.4679, grad_fn=<NllLossBackward0>)
128
acc: 0.328125
tensor(0.9902, grad_fn=<NllLossBackward0>)
256
acc: 0.421875
tensor(1.9568, grad_fn=<NllLossBackward0>)
384
acc: 0.4140625
tensor(1.5069, grad_fn=<NllLossBackward0>)
512
acc: 0.4921875
tensor(1.4375, grad_fn=<NllLossBackward0>)
640
acc: 0.4765625
tensor(1.5314, grad_fn=<NllLossBackward0>)
768
acc: 0.46875
tensor(1.2268, grad_fn=<NllLossBackward0>)
896
acc: 0.4609375


In [ ]:
model.eval()
with torch.no_grad():
    losses = 0.0
    total = 0
    corrects = 0
    tbie=[0,0,0,0,0]
    bie=[0,0,0,0,0]
    pre=[0,0,0,0,0]
    alls=[[0,0,0,0,0],[0,0,0,0,0],[0,0,0,0,0],[0,0,0,0,0],[0,0,0,0,0]]
    count=0
    for x, label in val_loader:
        x = x.to(device, dtype=torch.float32)          
        label = label.to(device, dtype=torch.long)
        batch = x.size(0)
        total += batch
        _,pred = model(x)
        loss = criterion(pred, label)
        losses += loss.item() * batch
        pred = torch.max(pred, dim=1)[1]
        for i in range(len(pred)):
            if int(pred[i])==int(label[i]):
                count+=1
        if total%128==0:
            print(loss)
            print(total)
            print("acc: "+str(count))
            count=0
        for i in range(len(pred)):
            cc=pred[i]
            cd=label[i]
            tbie[int(cd)]+=1
            alls[cc][cd]+=1
            if int(cc)==int(cd):
                corrects=corrects+1
                bie[int(cc)]+=1
    print("correct: "+str(corrects/total))
    for z in range(5):
        print("class "+str(z)+" correct "+str(bie[z]/tbie[z]))
        print("which are "+str(alls[z]))
    print(bie)

NameError: ignored

In [ ]:
test_dir = os.path.abspath('aptos2019-blindness-detection/test_images')
test_paths = glob.glob(os.path.join(test_dir, "*.png"))
filenames = []
preds = []
with torch.no_grad():
    model.eval()
    for path in test_paths:
        filename = os.path.basename(path)
        filenames.append(filename)
        img = Image.open(path)
        img = val_transform(img)
        img=img.unsqueeze(0)
        img = img.to(device)
        _,pred = model(img)
        pred = torch.max(pred, dim=1)[1]
        preds.append(int(pred))

output_dict = {
    "filename":filenames,
    "pred":preds
}

import pandas as pd
output = pd.DataFrame(output_dict)
output.to_csv("./submission.csv", index=False, encoding="utf-8")

NameError: ignored